In [1]:
from scipy import signal
import numpy as np
from scipy.fftpack import fft, ifft
import pandas as pd

import matplotlib
matplotlib.use('nbagg')
from pylab import rcParams
rcParams['figure.figsize'] = 9, 4
import matplotlib.pyplot as plt
import scipy

%load_ext autoreload
%autoreload 2
import sys
import copy

from pyha import Hardware, simulate, sims_close, Complex, resize, hardware_sims_equal
from pathlib import Path
from data import load_iq

def imshow(im):
    from skimage.exposure import exposure
    p2, p98 = np.percentile(im, (2, 98))
    im = exposure.rescale_intensity(im, in_range=(p2, p98))

    
    plt.imshow(im, interpolation='nearest', aspect='auto', origin='lower')
    plt.tight_layout()
    plt.show()
    
    
def snr(pure, noisy):
    sig_pow = np.mean(np.abs(pure))
    error = np.array(pure) - np.array(noisy)
    err_pow = np.mean(np.abs(error))
    
    snr_db = 20*np.log10(sig_pow/err_pow)
    return snr_db

from under_construction.fft.bit_reversal_fftshift import bit_reversed_indexes
from under_construction.fft.fft_core import R2SDFSFIX, R2SDF
from under_construction.fft.packager import DataWithIndex, unpackage, package

def show_freqz(taps):
    w, h = scipy.signal.freqz(taps)
    fig, ax1 = plt.subplots(1, 1)
    plt.title('Digital filter frequency response')
    ax1.plot(w / np.pi, 20 * np.log10(abs(h)), 'b')
    ax1.set_ylabel('Amplitude [dB]', color='b')
    ax1.set_xlabel('Frequency')
    plt.grid()
    ax2 = ax1.twinx()
    angles = np.unwrap(np.angle(h))
    ax2.plot(w / np.pi, angles, 'g')
    ax2.set_ylabel('Angle (radians)', color='g')
    ax2.axis('tight')
    plt.tight_layout()
    plt.show()
    
def show_plot():
    plt.tight_layout()
    plt.grid()
    if plt.gca().get_legend_handles_labels() != ([], []):
        plt.legend()
    plt.show()
    
def bit_reverse(x, n_bits):
    return int(np.binary_repr(x, n_bits)[::-1], 2)

def bit_reversed_indexes(N):
    return [bit_reverse(i, int(np.log2(N))) for i in range(N)]

from pyha import Hardware, simulate, sims_close, Complex, Sfix
from pyha.common.float import *
# Float.use_float16 = True
# Float.use_float16 = False

from mpmath import mp 
results = {}

# FIR tests

In [ ]:
class FIRFloat(Hardware):
    def __init__(self, taps):
        self.DELAY = 2

        self.TAPS = [Float(x) for x in np.array(taps).tolist()]
#         self.TAPS = np.array(taps).tolist()
        self.TAPS_ORIG = taps

        # registers
        self.acc = [Float()] * len(taps)
        self.mul = [Float()] * len(taps)
        # self.out = Float()

    def main(self, x):
        """ Transposed FIR structure """
        self.acc[0] = x * self.TAPS[-1]
        for i in range(1, len(self.acc)):
            self.mul[i] = x * self.TAPS[len(self.TAPS) - 1 - i]
            self.acc[i] = self.acc[i - 1] + self.mul[i]

        return self.acc[-1]

    def model_main(self, x):
        return signal.lfilter(self.TAPS_ORIG, [1.0], x)

In [ ]:
np.random.seed(0)
taps = signal.remez(128, [0, 0.1, 0.2, 0.5], [1, 0])
inp = np.random.uniform(-1, 1, 1024)

In [ ]:
Float.radix = 32
Float.default_exponent_bits = 3
Float.default_fractional_bits = 15
Float.use_float16 = False
dut = FIRFloat(taps)
sims = simulate(dut, inp, input_types=[Float()], simulations=['MODEL', 'PYHA'])
results['Float64'] = np.array([float(x) for x in sims['MODEL']])
results['Radix32'] = np.array([float(x) for x in sims['PYHA']])

Float.radix = 16
Float.default_exponent_bits = 4
Float.default_fractional_bits = 14
Float.use_float16 = False
dut = FIRFloat(taps)
sims = simulate(dut, inp, input_types=[Float()], simulations=['PYHA'])
results['Radix16'] = np.array([float(x) for x in sims['PYHA']])

print('Running float16')
Float.use_float16 = True
Float.radix = 16
Float.default_exponent_bits = 1
Float.default_fractional_bits = 1
dut = FIRFloat(taps)
sims = simulate(dut, inp, input_types=[Float()], simulations=['PYHA'])
results['Float16'] = np.array([float(x) for x in sims['PYHA']])


In [ ]:
for key, val in results.items():
    plt.magnitude_spectrum(val, scale='dB', label=key)
plt.legend()
plt.grid()
plt.show()

for key, val in results.items():
    plt.plot(val, label=key)
plt.legend()
plt.grid()
plt.show()

In [ ]:
for key, val in results.items():
    plt.plot(results['Float64'] - val, label=key)
plt.legend()
plt.grid()
plt.show()

for key, val in results.items():
    print(f'{key} SNR: {snr(results["Float64"], val)} dB')
#     plt.plot(results['Float64'] - val, label=key)

In [ ]:
128 more input
58.958953403445484 - float16
61.62623593672434 - R16

In [ ]:
128 tap
58.94898493380129 - float16
62.63310859634656 - R16
diff = 3.684123663

In [ ]:
64 tap
60.36905532922837 - float16
64.52083838567539 - R16
diff = 4.151783056

In [ ]:
69.83881625316633 - round()
64.19860087442991 - int()
63.47426287423282 - floor/ceil() ie always away from 0
47.395264674165475 - floor()

58.8762989679605 - int() R16
63.501908738503055 - R64